In [1]:
# --- Bibliotecas ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# --- Valores iniciales para la simulación ---
Subyacente = 21.77
Plazo = 90
Puntos_fwd = 4118
Precio_fwd = Subyacente + Puntos_fwd/10000 # 22.1818
Tasa_USD = 0.78413/100
Tasa_q = (360/Plazo)*np.log(1+Tasa_USD*Plazo/360)
Tasa_implicita_MXN = ((Precio_fwd/Subyacente)*(1+Tasa_USD*Plazo/360)-1)*(360/Plazo) # 0.08365307734496952
Tasa_r = (360/Plazo)*np.log(1+Tasa_implicita_MXN*Plazo/360)
Volatilidad = 20.00/100
Tamaño_paso_dias = 1.00
Nocional = 10000000
df_TIIE3m = 0.0713
Prob_cond = 0.03/100

## Construcción del CVA a partir de una simulación MonteCarlo

Creamos por partes la funcionalidad de la simulación que generará una simulación del valor del CVA(EPE) dados ciertos valores iniciales.

In [7]:
# --- Inicializamos la simulación utilizando el valor del subyacente como base ---
Tray_tiempo = pd.DataFrame([Subyacente]*1000, columns=[0])

# --- Realizamos la simulación utilizando la formula recursiva ---
for i in range(Plazo):
    Tray_tiempo[i+Tamaño_paso_dias] = Tray_tiempo[Tray_tiempo.columns[i]] * \
    np.exp( (Tasa_r-Tasa_q-0.5*Volatilidad**2)*(Tamaño_paso_dias/360) + Volatilidad*np.sqrt(Tamaño_paso_dias/360)*np.random.normal(size=1000) )

In [8]:
Tray_tiempo

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,21.77,21.824447,21.896347,21.561477,21.730049,21.659559,21.526336,21.394781,21.314590,21.103881,...,21.481826,21.470077,21.518478,21.750327,21.745342,21.627237,21.714627,21.665278,21.780043,21.745629
1,21.77,21.956427,22.075693,22.237915,22.199736,21.912034,22.100893,21.860325,22.158033,22.189505,...,23.405426,23.602421,23.467182,23.416592,23.623492,23.497967,23.418594,23.220329,23.345695,23.469520
2,21.77,21.818315,22.039706,21.991643,21.915226,22.066558,22.367118,22.777364,23.028587,23.026336,...,23.121002,23.742376,23.779243,23.652932,23.791858,23.555456,23.595462,23.643809,23.586883,23.519929
3,21.77,22.137695,22.771251,22.387252,22.462351,22.550376,22.721723,22.656555,22.746087,22.750426,...,24.809773,25.017575,25.328777,25.552991,25.401874,25.656568,25.867794,26.081357,25.774808,25.718109
4,21.77,22.106764,22.496523,22.564994,22.423954,22.282981,22.324446,22.316645,22.522198,22.839185,...,22.071274,22.473915,22.641773,22.217856,21.781403,21.542212,21.297308,20.760318,20.688367,20.379193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,21.77,21.615288,21.294196,21.792317,21.953036,21.815680,21.923802,22.203955,21.974160,21.988136,...,22.917408,23.117379,23.093481,22.948504,23.058599,22.487033,22.318539,22.612109,22.615968,22.746240
996,21.77,21.652865,21.735133,21.921211,21.707435,21.605225,21.537230,21.487784,21.813283,22.036909,...,23.122371,23.117545,22.905640,22.782573,23.201545,23.276575,23.116647,23.059481,22.885249,22.881937
997,21.77,21.999013,21.617530,21.414312,21.417690,21.201274,21.021191,21.047750,21.012895,20.936887,...,19.782340,19.807457,19.745308,19.735850,19.855817,20.149508,20.375412,20.459864,20.581590,20.587260
998,21.77,21.622473,21.531628,21.653747,21.898766,22.234479,22.547916,22.327830,22.519287,22.520429,...,23.131062,23.333155,23.213679,22.997883,23.068828,23.262238,23.631364,23.826730,24.130889,24.347699


In [9]:
Tray_tiempo[90].mean()

22.108697606213713

In [10]:
# --- Obtenemos los valores para la simulación con Nocional de valor 10,000,000 ---
mtm = pd.DataFrame()

for i in range(0, Plazo+1):
    mtm[i] = Nocional*( Tray_tiempo[i]*np.exp((Tasa_r-Tasa_q)*(Plazo-Tray_tiempo.columns[i])/360)-Precio_fwd )\
    * np.exp(-Tasa_r*(Plazo-Tray_tiempo.columns[i])/360)

In [11]:
mtm

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,-3.479937e-08,4.981748e+05,1.170550e+06,-2.217054e+06,-5.798077e+05,-1.328708e+06,-2.703837e+06,-4.062371e+06,-4.908266e+06,-7.057073e+06,...,-6.583170e+06,-6.746878e+06,-6.309196e+06,-4.037264e+06,-4.133321e+06,-5.360502e+06,-4.532921e+06,-5.072649e+06,-3.971304e+06,-4.361714e+06
1,-3.479937e-08,1.815417e+06,2.960576e+06,4.534535e+06,4.108279e+06,1.191375e+06,3.031241e+06,5.846645e+05,3.511135e+06,3.780046e+06,...,1.264906e+07,1.457285e+07,1.317488e+07,1.262322e+07,1.464613e+07,1.334518e+07,1.250564e+07,1.047719e+07,1.168487e+07,1.287720e+07
2,-3.479937e-08,4.369659e+05,2.601394e+06,2.076472e+06,1.268496e+06,2.733759e+06,5.688631e+06,9.738508e+06,1.220115e+07,1.213361e+07,...,9.805381e+06,1.597216e+07,1.629501e+07,1.498631e+07,1.632961e+07,1.392001e+07,1.427420e+07,1.471180e+07,1.409670e+07,1.338129e+07
3,-3.479937e-08,3.624588e+06,9.902849e+06,6.025076e+06,6.729520e+06,7.562997e+06,9.228204e+06,8.532596e+06,9.381192e+06,9.379381e+06,...,2.668978e+07,2.872193e+07,3.178799e+07,3.398441e+07,3.242802e+07,3.492930e+07,3.699603e+07,3.908622e+07,3.597547e+07,3.536309e+07
4,-3.479937e-08,3.315876e+06,7.160829e+06,7.799134e+06,6.346261e+06,4.893992e+06,5.262689e+06,5.139626e+06,7.146291e+06,1.026541e+07,...,-6.898460e+05,3.289752e+06,4.922043e+06,6.374217e+05,-3.772754e+06,-6.210675e+06,-8.705843e+06,-1.412186e+07,-1.488783e+07,-1.802607e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-3.479937e-08,-1.589376e+06,-4.839437e+06,8.697562e+04,1.645892e+06,2.296124e+05,1.263571e+06,4.014760e+06,1.675678e+06,1.769901e+06,...,7.769842e+06,9.723277e+06,9.438440e+06,7.942946e+06,8.997817e+06,3.236709e+06,1.505804e+06,4.395252e+06,4.387764e+06,5.644404e+06
996,-3.479937e-08,-1.214328e+06,-4.385024e+05,1.373481e+06,-8.055309e+05,-1.871046e+06,-2.595090e+06,-3.134028e+06,6.977560e+04,2.256769e+06,...,9.819065e+06,9.724935e+06,7.560308e+06,6.283852e+06,1.042712e+07,1.113145e+07,9.486365e+06,8.868771e+06,7.080515e+06,7.001365e+06
997,-3.479937e-08,2.240453e+06,-1.612290e+06,-3.685927e+06,-3.697563e+06,-5.903094e+06,-7.746054e+06,-7.526422e+06,-7.919838e+06,-8.724079e+06,...,-2.357470e+07,-2.337018e+07,-2.403819e+07,-2.417941e+07,-2.302652e+07,-2.013650e+07,-1.792420e+07,-1.712627e+07,-1.595557e+07,-1.594540e+07
998,-3.479937e-08,-1.517659e+06,-2.469657e+06,-1.296104e+06,1.104204e+06,4.409863e+06,7.493312e+06,5.251282e+06,7.117230e+06,7.083461e+06,...,9.905957e+06,1.188066e+07,1.064024e+07,8.436672e+06,9.100094e+06,1.098808e+07,1.463320e+07,1.654093e+07,1.953665e+07,2.165899e+07


In [12]:
# --- Calculamos el EPE ---
EPE = pd.DataFrame()

for i in range(1, Plazo+1):
    EPE.loc[0, i] = mtm.loc[mtm[i]>0, i].mean()
    
EPE

,1,2,3,4,5,6,7,8,9,10,...,81,82,83,84,85,86,87,88,89,90
0,1.815887e+06,2.560592e+06,3.181399e+06,3.731910e+06,3.933467e+06,4.390281e+06,4.900237e+06,5.343810e+06,5.686636e+06,6.003659e+06,...,1.632726e+07,1.658168e+07,1.659350e+07,1.671181e+07,1.701612e+07,1.705438e+07,1.712438e+07,1.728950e+07,1.735883e+07,1.746504e+07


In [13]:
# Dataframe con el valor de la TIIE
df_TIIE = pd.DataFrame()

for i in range(1, Plazo+1):
    df_TIIE.loc[0, i] = 1/(1+df_TIIE3m*91/360)**(mtm.columns[i]/91)
    
df_TIIE

,1,2,3,4,5,6,7,8,9,10,...,81,82,83,84,85,86,87,88,89,90
0,0.999804,0.999607,0.999411,0.999215,0.999019,0.998823,0.998627,0.998431,0.998235,0.998039,...,0.984226,0.984033,0.98384,0.983647,0.983454,0.983261,0.983068,0.982875,0.982682,0.982489


In [14]:
# Dataframe con el valor de la probabilidad condicional
prob_cond = pd.DataFrame()

for i in range(1, Plazo+1):
    prob_cond.loc[0, i] = Prob_cond
    
prob_cond

,1,2,3,4,5,6,7,8,9,10,...,81,82,83,84,85,86,87,88,89,90
0,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,...,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003


In [18]:
# Calculo final del CVA dados los valores del EPE, la probabilidad condicional y el valor de TIIE
CVA = (EPE.T * df_TIIE.T * prob_cond.T).sum().item()
CVA

317046.61305156053

## Creación de una función que calcula el CVA usando los pasos anteriores

Basandonos de lo anteriormente desarrollado, podemos crear una función que genera valores del CVA por medio de la simulación montecarlo.

In [2]:
def cva_montecarlo(Subyacente, Plazo, Puntos_fwd, Tasa_USD, Volatilidad, Tamaño_paso_dias, Nocional, df_TIIE3m, Prob_cond):
    Precio_fwd = Subyacente + Puntos_fwd/10000 # 22.1818
    Tasa_q = (360/Plazo)*np.log(1+Tasa_USD*Plazo/360)
    Tasa_implicita_MXN = ((Precio_fwd/Subyacente)*(1+Tasa_USD*Plazo/360)-1)*(360/Plazo) # 0.08365307734496952
    Tasa_r = (360/Plazo)*np.log(1+Tasa_implicita_MXN*Plazo/360)

    # --- Inicializamos la simulación utilizando el valor del subyacente como base ---
    Tray_tiempo = pd.DataFrame([Subyacente]*1000, columns=[0])
    # --- Realizamos la simulación utilizando la formula recursiva ---
    for i in range(Plazo):
        Tray_tiempo[i+Tamaño_paso_dias] = Tray_tiempo[Tray_tiempo.columns[i]] * \
        np.exp( (Tasa_r-Tasa_q-0.5*Volatilidad**2)*(Tamaño_paso_dias/360) + Volatilidad*np.sqrt(Tamaño_paso_dias/360)*np.random.normal(size=1000) )
        
    # --- Obtenemos los valores para la simulación con Nocional de valor x ---
    mtm = pd.DataFrame()
    for i in range(0, Plazo+1):
        mtm[i] = Nocional*( Tray_tiempo[i]*np.exp((Tasa_r-Tasa_q)*(Plazo-Tray_tiempo.columns[i])/360)-Precio_fwd )\
        * np.exp(-Tasa_r*(Plazo-Tray_tiempo.columns[i])/360)
        
    # --- Calculamos el EPE ---
    EPE = pd.DataFrame()
    for i in range(1, Plazo+1):
        EPE.loc[0, i] = mtm.loc[mtm[i]>0, i].mean()
        
    # Dataframe con el valor de la TIIE
    df_TIIE = pd.DataFrame()
    for i in range(1, Plazo+1):
        df_TIIE.loc[0, i] = 1/(1+df_TIIE3m*91/360)**(mtm.columns[i]/91)
    
    # Dataframe con el valor de la probabilidad condicional
    prob_cond = pd.DataFrame()
    for i in range(1, Plazo+1):
        prob_cond.loc[0, i] = Prob_cond
        
    # Calculo final del CVA dados los valores del EPE, la probabilidad condicional y el valor de TIIE
    CVA = (EPE.T * df_TIIE.T * prob_cond.T).sum().item()
    return CVA

In [5]:
# --- Dados ciertos valores iniciales para la simulación podemos observar como se comporta el valor del CVA---
cva_montecarlo(Subyacente = 21.77, Plazo = 90, Puntos_fwd = 4118, Tasa_USD = 0.0078413\
               , Volatilidad = 0.2, Tamaño_paso_dias = 1.00, Nocional = 10000000, df_TIIE3m = 0.0713, Prob_cond = 0.0003)

309300.9707648988

In [6]:
# Realizamos el ejercicio de la simulación unas 1000 veces y observamos el promedio del valor del CVA
sims_mc = []
for _ in range(1000):
    cva_temp = cva_montecarlo(Subyacente = 21.77, Plazo = 90, Puntos_fwd = 4118, Tasa_USD = 0.0078413\
               , Volatilidad = 0.2, Tamaño_paso_dias = 1.00, Nocional = 10000000, df_TIIE3m = 0.0713, Prob_cond = 0.0003)
    sims_mc.append(cva_temp)

In [8]:
sims_mc[:10]

[321375.05533737456,
 321732.3883025724,
 323652.68190393323,
 328997.6208519425,
 330033.98382076086,
 311062.5757024479,
 332490.9196846857,
 326808.18588579923,
 331589.6935364958,
 343028.2033043073]

In [10]:
print("El valor del CVA después de 1000 simulaciones es en promedio: ",  np.mean(sims_mc))

El valor del CVA después de 1000 simulaciones es en promedio:  324887.47629300336
